<a href="https://colab.research.google.com/github/insyspo/Lattes_OpenAlex/blob/main/ISSI_2025_authors_Lattes_OpenAlex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Research Entities Disambiguation Methodology Tested on Brazilian Researchers Database


# Authors from Lattes with DOIs

In [ ]:
%%bigquery --project = insyspo

 SELECT a.IDLattes, c.author_id, COUNT(DISTINCT(a.DOI)) AS dois
      FROM (
        SELECT  IDLattes, DOI
        FROM `insyspo.projectdb_lattes.artigos`
        WHERE DOI IS NOT NULL
      )

# DOIs present in publications in Lattes

In [ ]:
%%bigquery --project = insyspo

    SELECT IDLattes, author_id, dois,
      ROW_NUMBER() OVER (PARTITION BY IDLattes ORDER BY dois DESC) AS r
    FROM (
      SELECT a.IDLattes, c.author_id, COUNT(DISTINCT(a.DOI)) AS dois
      FROM (
        SELECT  IDLattes, DOI
        FROM `insyspo.projectdb_lattes.artigos`
        WHERE DOI IS NOT NULL
      )

# Authorships in OpenAlex with the DOIs

In [ ]:
%%bigquery --project = insyspo

CREATE OR REPLACE TABLE insyspo.projectdb_lattes.ids_match_doi AS (
  SELECT *
  FROM (
    SELECT IDLattes, author_id, dois,
      ROW_NUMBER() OVER (PARTITION BY IDLattes ORDER BY dois DESC) AS r
    FROM (
      SELECT a.IDLattes, c.author_id, COUNT(DISTINCT(a.DOI)) AS dois
      FROM (
        SELECT  IDLattes, DOI
        FROM `insyspo.projectdb_lattes.artigos`
        WHERE DOI IS NOT NULL
      ) AS a
      JOIN `insyspo.publicdb_openalex_2024_04_rm.works` AS b
      ON a.DOI = b.doi
      JOIN `insyspo.publicdb_openalex_2024_04_rm.works_authorships` AS c
      ON b.id = c.work_id
      GROUP BY a.IDLattes, c.author_id
    )
    ORDER BY IDLattes, r
  )
  WHERE r <= 10
);

# Alternative names for authors in OpenAlex

In [ ]:
%%bigquery --project = insyspo

CREATE OR REPLACE TABLE insyspo.projectdb_lattes.names_match_doi AS (
  SELECT ids.*, authors.display_name_alternative AS display_name, autores.Nome
  FROM insyspo.projectdb_lattes.ids_match_doi AS ids
  JOIN insyspo.publicdb_openalex_2024_04_rm.authors_display_name_alternatives AS authors
  ON ids.author_id = authors.author_id
  JOIN insyspo.projectdb_lattes.geraisIDNome AS autores
  ON ids.IDLattes = autores.IDLattes
)

# Distance metrics

In [1]:
def names_distance_ordered(name_a,name_b):
  sa = split_name(name_a)
  sb = split_name(name_b)

  # Análise da lista maior e menor para a criação de pares ordenados na matriz
  if len(sa) >= len(sb):
    major = sa
    minor = sb
  else:
    major = sb
    minor = sa
  gap = len(major) - len(minor)

  not_found = 1.0
  D = np.ones((len(major), len(minor)))*(not_found)

  for i,ia in enumerate(major):
    for j,ib in enumerate(minor):

      # Pares ordenados proibitivos vão continuar com o valor de distância igual a not_found
      if i > (j+gap) or j > i:
        pass
      else:
        if len(ia) >=2 or len(ib) >= 2:
          rd = Lv.distance(ia,ib)/len(ia)
          #print(ia,ib,rd)
          D[i,j] = rd
          #print(ia,ib,Lv.distance(ia,ib),rd)
        if len(ia)==1 or len(ib)==1:
          if ia[0] == ib[0]:
            #print(ia,ib,0)
            D[i,j] = 0
    #print(D)

  dn = D.min(axis = 0).sum()
  return dn

# Calculation of distance metrics

In [ ]:
%%bigquery df --project = insyspo

SELECT *
FROM `insyspo.projectdb_lattes.match_lattes_openalex_filtered`

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df['distance_names'] = df.apply(lambda x: names_distance_ordered(x['display_name'],x['Nome']),axis=1)

In [ ]:
pandas_gbq.to_gbq(df,destination_table='projectdb_lattes.match_lattes_openalex_distances',
                  project_id='insyspo',if_exists = 'replace')

100%|██████████| 1/1 [00:00<00:00, 665.55it/s]


# Sort by the most likely match

In [ ]:
%%bigquery df --project = insyspo

CREATE OR REPLACE TABLE projectdb_lattes.match_lattes_openalex_distances_ AS (
  SELECT IDLattes, author_id,
    display_name,
    Nome,
    distance_names,
    ROW_NUMBER() OVER (PARTITION BY IDLattes ORDER BY distance_names ASC) AS seq
  FROM `insyspo.projectdb_lattes.match_lattes_openalex_distances`
  WHERE seq = 1 AND distance_names < 0.5
  ORDER BY distance_names DESC, IDLattes
)

Query is running:   0%|          |

## Cruzamento final das três etapas